<a href="https://colab.research.google.com/github/FuzzilyDeveloper/ML_Project/blob/master/Copy_of_tsne_exact_s5e2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install Qiskit-Aer with GPU support
!pip install qiskit==1.2 qiskit-machine-learning qiskit-aer-gpu pandas numpy scikit-learn scipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.3/202.3 MB 5.9 MB/s eta 0:00:00


In [3]:
!pip install qiskit-algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, ZFeatureMap
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_aer import AerSimulator
from qiskit.primitives import StatevectorEstimator
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.manifold import TSNE
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [5]:


# Load data
train_path = '/content/drive/MyDrive/playground-series-s5e2/train.csv'
test_path = '/content/drive/MyDrive/playground-series-s5e2/test.csv'
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [6]:


# Downsample training data
train_sample_size = 1500
df_train = df_train.sample(n=train_sample_size, random_state=18)

In [7]:
df_train

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
109306,109306,Adidas,Polyester,Small,10.0,Yes,No,Messenger,Red,28.095995,90.56022
47070,47070,Adidas,Nylon,Medium,3.0,No,Yes,Tote,Pink,14.516500,43.70482
113574,113574,Nike,Nylon,Small,10.0,Yes,No,Tote,Pink,13.837293,68.43588
217205,217205,Under Armour,Polyester,Medium,10.0,No,Yes,Tote,Green,23.782329,18.51944
129778,129778,Jansport,Nylon,Small,10.0,No,Yes,Backpack,Blue,21.720912,86.73641
...,...,...,...,...,...,...,...,...,...,...,...
100266,100266,Nike,Leather,Small,6.0,Yes,Yes,Backpack,Gray,6.654934,22.43816
329,329,Under Armour,Polyester,Large,8.0,No,No,Backpack,Gray,29.724672,27.15302
242420,242420,Under Armour,NaN,Large,1.0,Yes,Yes,Tote,Red,19.639528,39.42773
106019,106019,Adidas,Nylon,Large,10.0,Yes,Yes,Messenger,Blue,8.270824,60.08551


In [8]:

df_train_split, df_val = train_test_split(df_train, test_size=0.2, random_state=15)

In [9]:
df_train_split

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
132958,132958,Adidas,Nylon,Medium,5.0,No,No,Backpack,Gray,27.047817,37.22601
207982,207982,Under Armour,Nylon,Large,10.0,Yes,Yes,Backpack,Blue,26.243879,90.90401
259055,259055,Adidas,Canvas,Large,3.0,NaN,Yes,Messenger,Red,16.622632,32.93455
201614,201614,Jansport,Leather,Medium,1.0,No,Yes,Messenger,Green,14.897876,93.64163
153306,153306,Jansport,Canvas,Medium,3.0,Yes,Yes,Tote,Gray,24.851521,98.62946
...,...,...,...,...,...,...,...,...,...,...,...
70409,70409,Nike,Canvas,Large,1.0,Yes,Yes,Backpack,Blue,10.875415,126.44895
279982,279982,Jansport,Nylon,Large,2.0,No,No,Tote,Pink,15.934833,46.97287
73662,73662,Puma,Leather,Medium,3.0,No,Yes,Tote,Pink,21.051083,38.22934
209467,209467,Puma,Canvas,Medium,7.0,Yes,No,NaN,Red,24.633984,58.90467


In [10]:
df_val

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
48669,48669,Nike,Leather,Large,1.0,Yes,No,Backpack,Blue,5.839575,114.06716
248391,248391,Under Armour,Leather,Small,10.0,NaN,NaN,Backpack,Green,12.295914,107.14008
115059,115059,Under Armour,Canvas,Large,2.0,Yes,No,Backpack,Black,22.138422,27.54372
279314,279314,Puma,NaN,Large,8.0,NaN,Yes,Tote,Red,NaN,150.00000
202993,202993,Adidas,Leather,Large,3.0,No,Yes,Messenger,Black,16.929313,69.00305
...,...,...,...,...,...,...,...,...,...,...,...
58061,58061,Jansport,Nylon,Large,3.0,No,No,Messenger,Black,12.448131,69.14138
188816,188816,Puma,Canvas,Large,10.0,No,No,Messenger,Black,14.935507,34.87877
179202,179202,Puma,Nylon,Small,6.0,Yes,No,Tote,Blue,11.634274,139.28294
75615,75615,Jansport,Polyester,Medium,6.0,Yes,Yes,Backpack,Blue,17.676399,34.11219


In [11]:


# Split training data

# Manually select features
categorical_cols = ['Brand', 'Material','Size', 'Laptop Compartment',
                    'Waterproof', 'Style','Color','Compartments' ]
numerical_cols = ['Weight Capacity (kg)']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ]), categorical_cols),
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_cols)
    ])

In [12]:


# Prepare data
X_train = df_train_split[categorical_cols + numerical_cols]
y_train = df_train_split['Price']
X_val = df_val[categorical_cols + numerical_cols]
y_val = df_val['Price']
X_test = df_test[categorical_cols + numerical_cols]

# Apply preprocessing
# X_train_processed = preprocessor.fit_transform(X_train)
# X_val_processed = preprocessor.transform(X_val)
# X_test_processed = preprocessor.transform(X_test)


X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.fit_transform(X_val)
X_test_processed = preprocessor.fit_transform(X_test)

In [13]:
X_train_processed[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.32085609])

In [14]:
X_val_processed[0]

array([ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -1.74870277])

In [15]:


# t-SNE dimensionality reduction for training and validation
X_train_val_processed = np.vstack([X_train_processed, X_val_processed])
train_idx = len(X_train_processed)
n_components = 5
# tsne = TSNE(n_components=n_components, method='barnes_hut', random_state=42, n_iter=300)
tsne = TSNE(n_components=n_components, method='exact', random_state=42, n_iter=300)
X_train_val_reduced = tsne.fit_transform(X_train_val_processed)

/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [16]:
X_train_val_processed[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.32085609])

In [17]:
X_train_val_reduced[0]

array([-3.711786  ,  0.26484808,  3.6786275 ,  0.03875982, -0.17440148],
      dtype=float32)

In [18]:


# Split back into training and validation
X_train_reduced = X_train_val_reduced[:train_idx]
X_val_reduced = X_train_val_reduced[train_idx:]

In [19]:

# Approximate t-SNE for test set using k-NN
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_processed, X_train_reduced)
X_test_reduced = knn.predict(X_test_processed)

print(f"t-SNE applied with {n_components} components (training/validation), k-NN for test set")

t-SNE applied with 5 components (training/validation), k-NN for test set


In [20]:

# Scale target
y_min, y_max = y_train.min(), y_train.max()

In [21]:

# Quantum circuit
num_qubits = n_components
feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)
ansatz = RealAmplitudes(num_qubits=num_qubits, reps=1)
circuit = QuantumCircuit(num_qubits)
circuit.compose(feature_map, inplace=True)
circuit.compose(ansatz, inplace=True)

In [22]:

# Simulator and QNN
aer_simulator = AerSimulator(method='statevector', device='CPU')
estimator = StatevectorEstimator()
qnn = EstimatorQNN(
    circuit=circuit,
    estimator=estimator,
    input_params=circuit.parameters[:num_qubits],
    weight_params=circuit.parameters[num_qubits:]
)

In [23]:

# Training setup
num_weights = len(qnn.weight_params)
initial_weights = np.random.random(num_weights)
current_weights = initial_weights.copy()
best_weights = current_weights.copy()
best_val_loss = float('inf')
patience = 2
patience_counter = 0

In [24]:







# Training loop
for epoch in range(2):
    batch_num = 0
    print(f"Epoch {epoch + 1}")
    start_idx = 0
    while start_idx < len(X_train_reduced):
        end_idx = min(start_idx + 50, len(X_train_reduced))
        X_batch = X_train_reduced[start_idx:end_idx]
        y_batch = y_train.iloc[start_idx:end_idx] if isinstance(y_train, pd.Series) else y_train[start_idx:end_idx]

        print(f"Training on batch {batch_num + 1}")
        y_batch_scaled = 2 * (y_batch - y_min) / (y_max - y_min) - 1

        result = minimize(
            lambda weights: np.mean((qnn.forward(X_batch, weights).flatten() - y_batch_scaled) ** 2) + 0.005 * np.sum(weights ** 2),
            current_weights,
            method='COBYLA',
            options={'maxiter': 20}
        )
        current_weights = result.x
        batch_num += 1
        start_idx += 50
        if batch_num >= 5:
            break

    val_predictions = qnn.forward(X_val_reduced, current_weights).flatten()
    val_scaled = 2 * (y_val - y_min) / (y_max - y_min) - 1
    val_loss = mean_squared_error(val_scaled, val_predictions)
    print(f"Validation Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_weights = current_weights.copy()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

Epoch 1
Training on batch 1
Training on batch 2
Training on batch 3
Training on batch 4
Training on batch 5
Validation Loss: 0.3751
Epoch 2
Training on batch 1
Training on batch 2
Training on batch 3
Training on batch 4
Training on batch 5
Validation Loss: 0.3701


In [25]:


# Training predictions
train_predictions = []
train_actual = []
start_idx = 0
while start_idx < len(X_train_reduced):
    end_idx = min(start_idx + 50, len(X_train_reduced))
    X_batch = X_train_reduced[start_idx:end_idx]
    y_batch = y_train.iloc[start_idx:end_idx] if isinstance(y_train, pd.Series) else y_train[start_idx:end_idx]
    y_pred = qnn.forward(X_batch, best_weights).flatten()
    y_pred_scaled = y_min + (y_pred + 1) * (y_max - y_min) / 2
    train_predictions.extend(y_pred_scaled)
    train_actual.extend(y_batch)
    start_idx += 50

mse = mean_squared_error(train_actual, train_predictions)
print(f"Training MSE: {mse:.4f}")

# Test predictions in chunks
test_predictions = []
batch_size = 10000  # Process test set in smaller chunks
start_idx = 0
while start_idx < len(X_test_reduced):
    end_idx = min(start_idx + batch_size, len(X_test_reduced))
    X_batch = X_test_reduced[start_idx:end_idx]
    y_pred = qnn.forward(X_batch, best_weights).flatten()
    y_pred_scaled = y_min + (y_pred + 1) * (y_max - y_min) / 2
    test_predictions.extend(y_pred_scaled)
    start_idx += batch_size

# Output results
print("\nSample Training Predictions (first 5):")
for i in range(min(5, len(train_predictions))):
    print(f"Actual: {train_actual[i]:.2f}, Predicted: {train_predictions[i]:.2f}")

df_test['Predicted_Price'] = test_predictions
print("\nSample Test Predictions (first 5 rows):")
print(df_test.head())

# Ensure submission has 200,000 rows
df_test_pred = df_test[['id', 'Predicted_Price']]
df_test_pred.columns = ['id', 'Price']
if len(df_test_pred) != 200000:
    raise ValueError(f"Submission has {len(df_test_pred)} rows, expected 200,000")
df_test_pred.to_csv('submission.csv', index=False)
print("\nTest predictions saved to 'submission.csv' with 200,000 rows")

Training MSE: 1587.1197

Sample Training Predictions (first 5):
Actual: 37.23, Predicted: 72.77
Actual: 90.90, Predicted: 79.69
Actual: 32.93, Predicted: 90.72
Actual: 93.64, Predicted: 63.15
Actual: 98.63, Predicted: 73.84

Sample Test Predictions (first 5 rows):
       id   Brand Material    Size  Compartments Laptop Compartment  \
0  300000    Puma  Leather   Small           2.0                 No   
1  300001    Nike   Canvas  Medium           7.0                 No   
2  300002  Adidas   Canvas   Large           9.0                 No   
3  300003  Adidas    Nylon   Large           1.0                Yes   
4  300004     NaN    Nylon   Large           2.0                Yes   

  Waterproof      Style  Color  Weight Capacity (kg)  Predicted_Price  
0         No       Tote  Green             20.671147        71.958176  
1        Yes   Backpack  Green             13.564105        73.461784  
2        Yes  Messenger   Blue             11.809799        83.384994  
3         No  Messen